In [1]:
import pandas as pd
import numpy as np
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (16, 9)
plt.style.use('ggplot')
from functools import reduce
import datetime
from scipy import stats
import seaborn as sns

In [3]:
path1="/Users/chrestoslogaras/Desktop/novibet/Casino games (1).csv"
games = pd.read_csv(path1)

In [4]:
games.rename(columns={"UserID": "UserProfileId"},inplace=True)
games['DATE'] = pd.to_datetime(games['Date']).dt.date

In [5]:
#Parameters for periods
freq='W'
start_length=14
label_period_days=7

# Get start and end dates of dataset
start_date = games['DATE'].min() + pd.Timedelta(start_length, unit="D")
end_date = games['DATE'].max() #- pd.Timedelta(label_period_days, unit="D")  -> if i wanted to exclude last period to use as a validation
# Get dates at desired interval
rng = pd.date_range(start=start_date, end=end_date, freq=freq)#datetime642
dates = rng.date

In [6]:
def generate_churn_labels(future):
    future['DidBuy'] = 1
    return future[['UserProfileId', 'DidBuy']]
def data_preprocess(games,cut_off):
   

    games.sort_values(by=['UserProfileId', 'Date'],inplace=True)

    #''' providers per player''' 
    pcur=games.groupby(['UserProfileId'], as_index=False)['Casino_Provider'].nunique()
    pcur.rename(columns = {'Casino_Provider':'platforms'}, inplace = True)
    #''' frequency per id''' 
    freq=games.groupby(['UserProfileId'], as_index=False)['Casino_Provider'].count()
    freq.rename(columns = {'Casino_Provider':'F'}, inplace = True)

    income=games[(games['Hold']>0)].groupby(['UserProfileId'], as_index=False)['Hold'].sum()
    income.rename(columns = {'Hold':'I'}, inplace = True)
    #''' profit per player (positive=profit, negative= loss, ''' 
    profit=games.groupby(['UserProfileId'], as_index=False)['Hold'].sum()
    profit.rename(columns = {'Hold':'P'}, inplace = True)
    #''' sum of ingame transactions per player (total monetary value)''' 
    games['absvalue']=games['Hold'].abs()
    absvalue=games.groupby(['UserProfileId'], as_index=False)['absvalue'].sum()
    #'''difference between games measured in days
    games.sort_values(['UserProfileId','Date'],inplace=True)
    
    
    games_occur=games.groupby('UserProfileId')['Date'].min().reset_index()
    games_occur['minDate'] = pd.to_datetime(games_occur['Date']).dt.date
    games_occur['cutoff'] = cut_off
    
    games_nq=games.groupby('UserProfileId')['Date'].nunique().reset_index()
    games_nq['nuniqueDate'] = games_nq['Date']
    
    games_mxdt=games.groupby('UserProfileId')['Date'].max().reset_index()
    games_mxdt['maxDate'] = pd.to_datetime( games_mxdt['Date']).dt.date  

    #''' average use of free bonus per player'''
    bonus=games.groupby(['UserProfileId'], as_index=False)['IsFreeSpinID'].mean()
    #''' average use of live games per player'''
    live=games.groupby(['UserProfileId'], as_index=False)['IsLiveID'].mean()    
    #''' merge all DataFrames into one'''
    dfss = [pcur, freq,absvalue,profit,games_occur,bonus,live,games_nq,games_mxdt]
    gameid = reduce(lambda  left,right: pd.merge(left,right,on=["UserProfileId"],how='outer'), dfss)
    #'''novibet profit over all ingame transactions per player
    gameid['Pt%']=gameid['P']/gameid['absvalue']
    gameid['age']=(gameid['maxDate']-gameid['minDate']).dt.days
    gameid.drop(axis=1,columns='P',inplace=True)
    #''' fill na Pt where player with single instance and 0 monetary outcome of game
    gameid['Pt%'].fillna(0,inplace=True)
    gameid['valpergame']=gameid['absvalue']/gameid['F']
    gameid.drop(axis=1,columns='absvalue',inplace=True)
    df1=gameid.copy()
    #df1.fillna(0,inplace=True)
 
    return df1

In [7]:
dset_list = []

#if we examined each period seperately and not in a recursive manner:
#for i in range(0,(len(dates)-1)):
#    if i==0:
#        gamess = games[games['DATE'] <= dates[i]]
#    else:
#        gamess = games[(games['DATE'] > dates[i-1])  & (games['DATE'] <= dates[i])]
#
#    
#    future = games[(games['DATE'] > dates[i])&\
#                   (games['DATE'] <= (dates[i] + pd.Timedelta(label_period_days, unit='D')))]
for i in range(0,(len(dates)-1)):
    if i==0:
        gamess = games[games['DATE'] <= dates[i]]
    else:
        a=games[(games['DATE'] > dates[i-1])  & (games['DATE'] <= dates[i])]['UserProfileId'].unique().tolist()
        gamess = games[(games['DATE'] <= dates[i])&(games['UserProfileId'].isin(a) ) ] 
        # i do not want clients who do not play again to keep appearing beacause the increase the class=0
        

    
    future = games[(games['DATE'] > dates[i])&\
                   (games['DATE'] <= (dates[i] + pd.Timedelta(label_period_days, unit='D')))]

    print(f"computing rfm features for predicting the churn in future period between {dates[i]} and {future['DATE'].max()}:")
    _observed = data_preprocess(gamess,dates[i])
    unique_scores = pd.DataFrame(future['UserProfileId'].unique(),columns = ['UserProfileId'])
    labels = generate_churn_labels(unique_scores)
    
    dset = _observed.merge(labels, on='UserProfileId', how='left').fillna(0) 
    dset_list.append(dset)

full_dataset = pd.concat(dset_list, axis=0)

computing rfm features for predicting the churn in future period between 2021-09-19 and 2021-09-26:


/Users/chrestoslogaras/opt/anaconda3/envs/py37/lib/python3.7/site-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)
/Users/chrestoslogaras/opt/anaconda3/envs/py37/lib/python3.7/site-packages/ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


computing rfm features for predicting the churn in future period between 2021-09-26 and 2021-10-03:
computing rfm features for predicting the churn in future period between 2021-10-03 and 2021-10-10:
computing rfm features for predicting the churn in future period between 2021-10-10 and 2021-10-17:
computing rfm features for predicting the churn in future period between 2021-10-17 and 2021-10-24:
computing rfm features for predicting the churn in future period between 2021-10-24 and 2021-10-31:


In [7]:
#Checking
#games[games['UserProfileId']==583236].sort_values(by=['Date'])
#full_dataset[full_dataset['DidBuy']==0]
#full_dataset[full_dataset['UserProfileId']==583003]

In [8]:
full_dataset.to_csv('out.zip')

full_dataset[full_dataset['UserProfileId']==104]
games[games['UserProfileId']==104]

In [8]:
full_dataset.DidBuy.value_counts()

1.0    551299
0.0     71560
Name: DidBuy, dtype: int64

'''Warning: Players that first start playing in last period are exluded in this analysis'''

a=full_dataset['UserProfileId'].unique().tolist()

b=games['UserProfileId'].unique().tolist()

b_not_a = set(b).difference(a)

len(b_not_a)